In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [15]:
## Definicion de la arquitectura del modelo
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(28*28, 20) #hidden layer 1
        self.relu = nn.ReLU() #funcion de activacion relu
        self.fc2 = nn.Linear(20, 10)  #hidden layer 2
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = x.view(-1, 28*28) #el -1 hace que esa primera dimension se autoajuste en funcion de las otras
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


In [16]:
# Carga de los datos
transform = transforms.ToTensor()
train_dataset = datasets.MNIST("./data", train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST("./data", train=False, download=True, transform=transform)

In [17]:
# Procesado de los datos
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [18]:
# Configuración del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network = NeuralNetwork().to(device)

In [26]:
# Optimozador y funcion de perdida
optimizer = optim.Adam(network.parameters(), lr=0.01, betas=(0.9, 0.99))
loss_criterion = nn.CrossEntropyLoss()

In [27]:
# Funcion de entrenamiento
def train(network, device, train_loader, optimizer, epoch):
    network.train()
    for batch_id, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        output = network(data)
        loss = loss_criterion(output, label)
        loss.backward()
        optimizer.step()
        if batch_id % 100 == 0:
            print(f"Epoch {epoch} [{batch_id * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():6f}")

In [28]:
# Funcion de testeo
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_criterion(output, target).item()  # Suma de la pérdida
            pred = output.argmax(dim=1, keepdim=True)        # Predicción: índice de la clase mayor probabilidad
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")

In [29]:
for epoch in range(1,2):
    train(network, device, train_loader, optimizer, epoch)
    test(network, device, test_loader)

Epoch 1 [0/60000] Loss: 1.490152
Epoch 1 [12800/60000] Loss: 1.527799
Epoch 1 [25600/60000] Loss: 1.506570
Epoch 1 [38400/60000] Loss: 1.499510
Epoch 1 [51200/60000] Loss: 1.538612

Test set: Average loss: 0.0119, Accuracy: 9465/10000 (95%)

